<h1> Text Classification using TensorFlow on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using a CNN and custom Estimator 
<li> Training on Cloud ML Engine
<li> Deploying model
<li> Predicting with model
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%datalab project set -p $PROJECT

In [4]:
import tensorflow as tf
print tf.__version__

1.2.1


The idea is to look at the title of a newspaper article and figure out whether the article came from the New York Times or from TechCrunch. There are very sophisticated approaches that we can try, but for now, let's go with something very simple.

<h2> Data exploration and preprocessing in BigQuery </h2>
<p>
What does the Hacker News dataset look like?

In [5]:
%bq query
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
LIMIT 10

url,title,score
,"Ask HN: What's your speciality, and what's your ""FizzBuzz"" equivalent?",260
,Ask HN: Books with a high signal to noise ratio?,260
,Ask HN: Why can't I make as much as I make?,262
,Ask HN: Is it OK to submit job postings on HN?,11
,Ask HN: Can I help you be more awesome today? (No strings. Inquire within.),11
,Show HN: Free and Open Source book to teach Firefox OS app development,11
http://empowerunited.com/,Could this be the solution for the 99%?,11
https://github.com/Groundworkstech/Submicron,Deep-Submicron Backdoors,11
http://vancouver.en.craigslist.ca/van/roo/2035580033.html,Best Roommate Ad Ever,11
https://www.kickstarter.com/projects/carlosxcl/code-cards,"Show HN: Code Cards, Like Texas hold 'em for people who want to code",11


Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>. To ensure that the parsing works for all URLs of interest, I'll group by the source to make sure there are no weird names left. This was an iterative process.

In [6]:
query="""
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
GROUP BY
  source
ORDER BY num_articles DESC
LIMIT 10
"""

In [7]:
import google.datalab.bigquery as bq
df = bq.Query(query).execute().result().to_dataframe()
df

,source,num_articles
0,blogspot,41386
1,github,36525
2,techcrunch,30891
3,youtube,30848
4,nytimes,28787
5,medium,18422
6,google,18235
7,wordpress,17667
8,arstechnica,13749
9,wired,12841


Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for machine learning.

In [8]:
query="""
SELECT source, LOWER(REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ')) AS title FROM
(SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  title
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
)
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
"""
df = bq.Query(query + " LIMIT 10").execute().result().to_dataframe()
df.head()

,source,title
0,github,php bdd is now nice
1,github,mpv video player 0.2 release
2,github,show hn re-thinking the business card with dr...
3,github,update css js from chrome developer tool
4,github,simple way to start development with flask usi...


For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  A simple way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).
<p>
So, let's do that and save the results as CSV files.

In [9]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) = 0").execute().result().to_dataframe()
traindf.head()

,source,title
0,github,this guy just found out how to bypass adblocker
1,github,show hn dodo command line task management f...
2,github,show hn webservicemock mock out external ca...
3,github,magento category attributes dependency
4,github,write actionscript in swift whaa


In [10]:
traindf['source'].value_counts()

github        27445
techcrunch    23131
nytimes       21586
Name: source, dtype: int64

In [11]:
evaldf['source'].value_counts()

github        9080
techcrunch    7760
nytimes       7201
Name: source, dtype: int64

In [12]:
import os, shutil
DATADIR='data/txtcls'
shutil.rmtree(DATADIR, ignore_errors=True)
os.makedirs(DATADIR)
traindf.to_csv( os.path.join(DATADIR,'train.csv'), header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv( os.path.join(DATADIR,'eval.csv'), header=False, index=False, encoding='utf-8', sep='\t')

In [13]:
!head -3 data/txtcls/train.csv

github	this guy just found out how to bypass adblocker
github	show hn  dodo   command line task management for developers
github	show hn  webservicemock   mock out external calls for local development


In [14]:
!wc -l data/txtcls/*.csv

  24041 data/txtcls/eval.csv
  72162 data/txtcls/train.csv
  96203 total


In [15]:
%bash
gsutil cp data/txtcls/*.csv gs://${BUCKET}/txtcls/

Copying file://data/txtcls/eval.csv [Content-Type=text/csv]...
Copying file://data/txtcls/train.csv [Content-Type=text/csv]...
|
Operation completed over 2 objects/5.4 MiB.                                      


<h2> TensorFlow code </h2>

Please explore the code in this <a href="txtcls1/trainer">directory</a> -- <a href="txtcls1/trainer/model.py">model.py</a> contains the key TensorFlow model and <a href="txtcls1/trainer/task.py">task.py</a> has a main() that launches off the training job.

However, the following cells should give you an idea of what the model code does:

In [16]:
import tensorflow as tf
from tensorflow.contrib import lookup
from tensorflow.python.platform import gfile

print tf.__version__
MAX_DOCUMENT_LENGTH = 5  
PADWORD = 'ZYXW'

# vocabulary
lines = ['Some title', 'A longer title', 'An even longer title', 'This is longer than doc length']
lines = [line.lower() for line in lines]

# create vocabulary
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
vocab_processor.fit(lines)
with gfile.Open('vocab.tsv', 'wb') as f:
    f.write("{}\n".format(PADWORD))
    for word, index in vocab_processor.vocabulary_._mapping.iteritems():
      f.write("{}\n".format(word))
N_WORDS = len(vocab_processor.vocabulary_)
print '{} words into vocab.tsv'.format(N_WORDS)

# can use the vocabulary to convert words to numbers
table = lookup.index_table_from_file(
  vocabulary_file='vocab.tsv', num_oov_buckets=1, vocab_size=None, default_value=-1)
numbers = table.lookup(tf.constant(lines[0].split()))
with tf.Session() as sess:
  tf.tables_initializer().run()
  print "{} --> {}".format(lines[0], numbers.eval())   

1.2.1
12 words into vocab.tsv
some title --> [8 4]


In [17]:
!cat vocab.tsv

ZYXW
a
even
longer
title
this
doc
is
some
an
length
than
<UNK>


In [18]:
# string operations
titles = tf.constant(lines)
words = tf.string_split(titles)
densewords = tf.sparse_tensor_to_dense(words, default_value=PADWORD)
numbers = table.lookup(densewords)

# now pad out with zeros and then slice to constant length
padding = tf.constant([[0,0],[0,MAX_DOCUMENT_LENGTH]])
padded = tf.pad(numbers, padding)
sliced = tf.slice(padded, [0,0], [-1, MAX_DOCUMENT_LENGTH])

with tf.Session() as sess:
  tf.tables_initializer().run()
  print "titles=", titles.eval(), titles.shape
  print "words=", words.eval()
  print "dense=", densewords.eval(), densewords.shape
  print "numbers=", numbers.eval(), numbers.shape
  print "padding=", padding.eval(), padding.shape
  print "padded=", padded.eval(), padded.shape
  print "sliced=", sliced.eval(), sliced.shape 


titles= ['some title' 'a longer title' 'an even longer title'
 'this is longer than doc length'] (4,)
words= SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 3],
       [3, 4],
       [3, 5]]), values=array(['some', 'title', 'a', 'longer', 'title', 'an', 'even', 'longer',
       'title', 'this', 'is', 'longer', 'than', 'doc', 'length'], dtype=object), dense_shape=array([4, 6]))
dense= [['some' 'title' 'ZYXW' 'ZYXW' 'ZYXW' 'ZYXW']
 ['a' 'longer' 'title' 'ZYXW' 'ZYXW' 'ZYXW']
 ['an' 'even' 'longer' 'title' 'ZYXW' 'ZYXW']
 ['this' 'is' 'longer' 'than' 'doc' 'length']] (?, ?)
numbers= [[ 8  4  0  0  0  0]
 [ 1  3  4  0  0  0]
 [ 9  2  3  4  0  0]
 [ 5  7  3 11  6 10]] (?, ?)
padding= [[0 0]
 [0 5]] (2, 2)
padded= [[ 8  4  0  0  0  0  0  0  0  0  0]
 [ 1  3  4  0  0  0  0  0  0  0  0]
 [ 9  2  3  4  0  0  0  0  0  0  0]
 [ 5 

In [19]:
%bash
grep "^def" txtcls1/trainer/model.py

def init(hparams):
def save_vocab(trainfile, txtcolname, outfilename):
def read_dataset(hparams, prefix, batch_size=BATCH_SIZE):
def get_embedding(hparams, titles, embed_size):
def cnn_model(features, labels, mode, params):
def serving_input_fn():
def get_train(hparams):
def get_valid(hparams, batch_size):
def make_experiment_fn(output_dir, hparams):


Let's make sure the code works locally on a small dataset for a few steps.

In [ ]:
%bash
echo "bucket=${BUCKET}"
rm -rf outputdir
export PYTHONPATH=${PYTHONPATH}:${PWD}/txtcls1
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --train_steps=200

When I ran it, I got a 55% accuracy after a few steps. Because batchsize=32, 200 steps is essentially 6400 examples -- the full dataset is 72,000 examples, so this is not even the full dataset. And already, we are doing much better than random chance.
<p>
Once the code works in standalone mode, you can run it on Cloud ML Engine. You can monitor the job from the GCP console in the Cloud Machine Learning Engine section.  Since we have 72,000 examples and batchsize=32, train_steps=36,000 essentially means 16 epochs.

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/txtcls1/trained_model
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gsutil cp txtcls1/trainer/*.py $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/txtcls1/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC --runtime-version=1.2 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --train_steps=36000

Training finished with an accuracy of 73%.  Obviously, this was trained on a really small dataset and with more data will hopefully come even greater accuracy.

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [26]:
%bash
gsutil ls gs://${BUCKET}/txtcls1/trained_model/export/Servo/

gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/
gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/1515090618/


In [ ]:
%bash
MODEL_NAME="txtcls"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/txtcls1/trained_model/export/Servo/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

<h2> Use model to predict </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict which publication the article is more likely to run in. These are actual titles of articles in the New York Times, github, and TechCrunch on June 19.   These titles were not part of the training or evaluation datasets.

In [29]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'title': 'Supreme Court to Hear Major Case on Partisan Districts'.lower()
      },
      {
        'title': 'Furan -- build and push Docker images from GitHub to target'.lower()
      },
      {
        'title': 'Time Warner will spend $100M on Snapchat original shows and ads'.lower()
      },
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'txtcls', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

response={u'predictions': [{u'source': u'nytimes', u'prob': [0.8374465107917786, 0.04888102412223816, 0.11367242783308029], u'class': 0}, {u'source': u'github', u'prob': [0.012753208167850971, 0.987160325050354, 8.644349873065948e-05], u'class': 1}, {u'source': u'techcrunch', u'prob': [0.006752816028892994, 0.0005263009225018322, 0.992720901966095], u'class': 2}]}


As you can see, the trained model predicts that the Supreme Court article is 84% likely to come from New York Times. The Docker article is 99% likely to be from GitHub according to the service and the Time Warner one is 99% likely to be from TechCrunch.

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License